<a href="https://colab.research.google.com/github/lelongc/ro/blob/main/TEST" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Cài đặt Môi trường & Dependencies (Mất khoảng 10-15 phút)
# @markdown Bước này sẽ cài đặt TRELLIS.2 và các thư viện cần thiết. Vui lòng kiên nhẫn.

import os
import sys
from IPython.display import clear_output

# 1. Kiểm tra GPU
!nvidia-smi

# 2. Cài đặt các thư viện cơ bản
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install ninja imageio imageio-ffmpeg opencv-python-headless trimesh rembg
!pip install diffusers transformers accelerate  # Dùng để tạo ảnh từ text

# 3. Clone Repo TRELLIS.2
if not os.path.exists("TRELLIS.2"):
    !git clone -b main https://github.com/microsoft/TRELLIS.2.git --recursive

%cd TRELLIS.2

# 4. Cài đặt dependencies đặc thù (Phần này tốn thời gian nhất)
# Ép buộc sử dụng xformers thay vì flash-attn để tương thích tốt hơn với Colab T4
os.environ["ATTN_BACKEND"] = "xformers"
!pip install xformers

# Cài đặt các gói con (submodules)
# Lưu ý: Việc biên dịch các gói này tốn nhiều thời gian
print("Đang cài đặt cumesh, o-voxel, flexgemm... Vui lòng đợi...")
!pip install ./submodules/cumesh
!pip install ./submodules/o-voxel
!pip install ./submodules/flexgemm
!pip install ./submodules/nvdiffrast
!pip install ./submodules/nvdiffrec

# Cài đặt gói chính
!pip install .

clear_output()
print("✅ Đã cài đặt xong môi trường!")

Mon Dec 22 15:41:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# @title 2. Tạo ảnh "Stone Pet" từ mô tả của bạn
# @markdown Chúng ta sẽ dùng mô hình FLUX hoặc SDXL để tạo ảnh input chất lượng cao trước.

import torch
from diffusers import AutoPipelineForText2Image
from PIL import Image

# Cấu hình prompt từ yêu cầu của bạn
prompt = "Cute low-poly stone pet with water element, blue glowing eyes, isometric view, white background, 3d blender render style, high quality, single object"
negative_prompt = "shadow, complex background, text, watermark, blurry, noise, low quality, multiple angles"

# Load model tạo ảnh (Dùng model nhẹ để chạy nhanh trên Colab)
print("Đang tải model tạo ảnh...")
pipe = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/sdxl-turbo", # Dùng SDXL Turbo cho nhanh và đẹp
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

# Tạo ảnh
print(f"Đang vẽ: {prompt}")
image = pipe(prompt=prompt, num_inference_steps=2, guidance_scale=0.0).images[0]

# Lưu và hiển thị ảnh
input_image_path = "input_pet.png"
image.save(input_image_path)
display(image.resize((256, 256))) # Hiển thị ảnh nhỏ để xem trước
print("✅ Đã tạo ảnh input thành công!")

In [ ]:
# @title 3. Chạy TRELLIS.2 để tạo mô hình 3D (.GLB)
# @markdown Quá trình này sẽ biến bức ảnh trên thành file 3D.

import os
# Cấu hình môi trường để tiết kiệm VRAM
os.environ['OPENCV_IO_ENABLE_OPENEXR'] = '1'
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["ATTN_BACKEND"] = "xformers" # Quan trọng cho Colab T4

import cv2
import imageio
from PIL import Image
import torch
from trellis2.pipelines import Trellis2ImageTo3DPipeline
from trellis2.utils import render_utils
from trellis2.renderers import EnvMap
import o_voxel

# 1. Thiết lập ánh sáng môi trường (Environment Map)
# Tải một HDRI map mặc định nếu chưa có
if not os.path.exists('assets/hdri/forest.exr'):
    !mkdir -p assets/hdri
    !wget https://market-assets.fra1.cdn.digitaloceanspaces.com/market-assets/hdris/forest_slope/forest_slope_1k.exr -O assets/hdri/forest.exr

envmap = EnvMap(torch.tensor(
    cv2.cvtColor(cv2.imread('assets/hdri/forest.exr', cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB),
    dtype=torch.float32, device='cuda'
))

# 2. Load Pipeline TRELLIS.2
print("Đang tải model TRELLIS.2-4B (Rất nặng, có thể mất vài phút)...")
try:
    pipeline = Trellis2ImageTo3DPipeline.from_pretrained("microsoft/TRELLIS.2-4B")
    pipeline.cuda()
except Exception as e:
    print(f"Lỗi tải model: {e}")
    print("Nếu lỗi OOM (Out of Memory), hãy thử khởi động lại Runtime và chắc chắn bạn đang dùng GPU mạnh nhất có thể.")

# 3. Chạy Inference
print("Đang tạo mô hình 3D...")
try:
    image = Image.open(input_image_path)
    # Giảm seed để cố định kết quả hoặc random
    outputs = pipeline.run(image, seed=42)
    mesh = outputs[0]

    # Giảm lưới (Simplify) để nhẹ hơn cho Roblox
    mesh.simplify(1000000)

    # 4. Xuất file GLB
    print("Đang xuất file GLB...")
    glb = o_voxel.postprocess.to_glb(
        vertices            =   mesh.vertices,
        faces               =   mesh.faces,
        attr_volume         =   mesh.attrs,
        coords              =   mesh.coords,
        attr_layout         =   mesh.layout,
        voxel_size          =   mesh.voxel_size,
        aabb                =   [[-0.5, -0.5, -0.5], [0.5, 0.5, 0.5]],
        decimation_target   =   20000, # Giảm xuống 20k poly cho Roblox (Low-poly)
        texture_size        =   2048,  # Texture vừa đủ
        remesh              =   True,
        remesh_band         =   1,
        remesh_project      =   0,
        verbose             =   True
    )

    output_filename = "StonePet_Water.glb"
    glb.export(output_filename, extension_webp=True)

    print(f"✅ Xong! File {output_filename} đã sẵn sàng.")

    # 5. Tải xuống
    from google.colab import files
    files.download(output_filename)

except RuntimeError as e:
    if "out of memory" in str(e):
        print("❌ LỖI HẾT BỘ NHỚ (OOM): GPU T4 của Colab không đủ bộ nhớ cho độ phân giải mặc định.")
        print("Gợi ý: Hãy thử giảm độ phân giải input hoặc nâng cấp lên Colab Pro (A100).")
    else:
        print(f"Lỗi: {e}")